# WHISP pure Cloud Function

In [ ]:
# REPLACE WITH YOUR PROJECT!
PROJECT = 'your-project'
# Suggested compute region:
REGION = 'us-central1'

In [ ]:
!gcloud auth login --project {PROJECT} --billing-project {PROJECT} --update-adc

## Create the Cloud function and deploy it

In [ ]:
!mkdir whisper

Get the list of datasets from the WHISP GitHub repo.

In [ ]:
!curl https://raw.githubusercontent.com/forestdatapartnership/whisp/main/src/openforis_whisp/datasets.py --output whisper/datasets.py

# EEasify WHISP

In [ ]:
%%writefile whisper/easy_whisp.py

import google.auth
import ee
from typing import List

# First, initialize.
credentials, _ = google.auth.default(
    scopes=['https://www.googleapis.com/auth/earthengine']
)
ee.Initialize(credentials, project='forest-data-partnership', opt_url='https://earthengine-highvolume.googleapis.com')

from datasets import list_functions

def easy_whisp() -> List[ee.Image]:
    """Returns the stack as a list of images."""
    images_list = []
    for func in list_functions():
      try:
        image = func()
        images_list.append(image)
      except ee.EEException as e:
        logging.error(str(e))
    return images_list

In [ ]:
%%writefile whisper/main.py

import json
import ee
from flask import jsonify
import functions_framework
import logging
import requests
import google.auth
import google.cloud.logging
from google.api_core import retry
import concurrent.futures

from easy_whisp import easy_whisp

client = google.cloud.logging.Client()
client.setup_logging()


_WHISP_IMAGES = easy_whisp()


@retry.Retry()
def get_stats(region, image):
  """"""
  return image.reduceRegion(
      reducer=ee.Reducer.mean(), geometry=region, scale=10).getInfo()


@retry.Retry()
def get_whisp_stats(geojson):
  """"""
  region = ee.Geometry(geojson)
  whisp_stats = {}
  # Use ThreadPoolExecutor for parallel execution.
  with concurrent.futures.ThreadPoolExecutor(max_workers=len(_WHISP_IMAGES) + 5) as executor:
    future_to_image = {executor.submit(
        get_stats, region=region, image=img): img for img in _WHISP_IMAGES}
    for future in concurrent.futures.as_completed(future_to_image):
      img = future_to_image[future]
      try:
          image_stats = future.result()
          whisp_stats.update(image_stats)
      except ee.EEException as e:
          logging.error(f'{img} generated an exception: {e}')
  return whisp_stats


@functions_framework.http
def main(request):
  """"""
  credentials, _ = google.auth.default(
      scopes=['https://www.googleapis.com/auth/earthengine']
  )
  ee.Initialize(credentials, project='forest-data-partnership')
  try:
    replies = []
    request_json = request.get_json(silent=True)
    calls = request_json['calls']
    for call in calls:
      geo_json = json.loads(call[0])
      try:
        logging.info([geo_json])
        response = get_whisp_stats(geo_json)
        logging.info(response)
        replies.append(json.dumps(response))
      except Exception as e:
        logging.error(str(e))
        replies.append(json.dumps( { "errorMessage": str(e) } ))
    return jsonify(replies=replies, status=200, mimetype='application/json')
  except Exception as e:
    error_string = str(e)
    logging.error(error_string)
    return jsonify(error=error_string, status=400, mimetype='application/json')

In [ ]:
%%writefile whisper/requirements.txt
earthengine-api
flask
functions-framework
google-api-core
google-cloud-logging
requests

## Load WHISP example data

Here we will get the WHISP example data from GitHub and use it to test the Cloud Function.

In [ ]:
import json

In [ ]:
fc_list = !curl https://raw.githubusercontent.com/forestdatapartnership/whisp/main/tests/fixtures/geojson_example.geojson

In [ ]:
fc_obj = json.loads("\n".join(fc_list))

In [ ]:
features = fc_obj['features']

See https://code.earthengine.google.com/e7d74cb4694589fc8a2e9923404730b4

In [ ]:
feature = features[4]

In [ ]:
feature

In [ ]:
geoms = [f['geometry'] for f in features]

In [ ]:
geoms[4]

In [ ]:
json.dumps(geoms[4], separators=(',', ':'))

In [ ]:
import ee
ee.Initialize(project='forest-data-partnership')

In [ ]:
print(ee.Geometry(geoms[4]).getInfo())

## Deploy the Cloud Function

In [ ]:
!gcloud functions deploy 'whisper' \
  --gen2 \
  --region={REGION} \
  --project={PROJECT} \
  --runtime=python312 \
  --source='whisper' \
  --entry-point=main \
  --trigger-http \
  --no-allow-unauthenticated \
  --timeout=300s

## Test the deployed Cloud Function

In [ ]:
!gcloud auth print-identity-token

In [ ]:
import json

test_calls = [[json.dumps(g), 'foo_string', 'bar_string'] for g in geoms]
test_request = json.dumps({'calls': test_calls}, separators=(',', ':')).join("''")

In [ ]:
test_request

In [ ]:
responses = !curl -X POST https://{REGION}-{PROJECT}.cloudfunctions.net/whisper \
  -H "Authorization: bearer $(gcloud auth print-identity-token)" \
  -H "Content-Type: application/json" \
  -d {test_request}

### Inspect the output of the function

The keys are useful for making the SQL to use in BigQuery.

In [ ]:
print(len(responses))
response = responses[0]
response_json = json.loads(response)
replies = response_json['replies']
print(len(replies))
reply_0 = replies[0]
reply_0_json = json.loads(reply_0)
reply_0_json.keys()

## Create a remote connection in BQ

Follow https://cloud.google.com/bigquery/docs/remote-functions#create_a_remote_function to set up a connection to the Cloud Function deployed previously.  Once the connection is set up, create a function to use in queries:

```
CREATE OR REPLACE FUNCTION `forest-data-partnership.WHISP_DEMO.whisp`(geom STRING) RETURNS STRING
REMOTE WITH CONNECTION `forest-data-partnership.us-central1.whisp`
OPTIONS (
  endpoint = 'https://us-central1-forest-data-partnership.cloudfunctions.net/whisper',
  max_batching_rows = 1
)
```

Once that's done, you can use your EasyWHISP function in queries!  The keys extracted from the test response are useful for building the `SQL` that represents this query.  Note that the input table must have a geometry column and that the geometries are passed to the function as GeoJSON strings:

In [ ]:
SQL_TEMPLATE = [f"JSON_EXTRACT_SCALAR(json_data, '$.{key}') AS {key}," for key in reply_0_json.keys()]
SQL_TEMPLATE = ['SELECT', 'geometry,'] + SQL_TEMPLATE
SQL_TEMPLATE = SQL_TEMPLATE + [
    'FROM',
    '`forest-data-partnership.WHISP_DEMO.input_examples`,',
    'UNNEST([SAFE.PARSE_JSON(`forest-data-partnership.WHISP_DEMO`.whisp(ST_ASGEOJSON(geometry)))]) AS json_data']

print('\n'.join(SQL_TEMPLATE))

Take that `SQL` blob over to BigQuery and run it!